# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [3]:
# definir variable de usuario y ejecutar condicionalmente cualquier código que dependa del contexto de ejecución
# no borrar código, simplemente ejecutarlo o no con un "if"
# cada uno mantiene actualizada su parte del "if" y no toca la del otro

user = 'navia'
#user = 'roberto'

#modelo = 'hybrid' 
#modelo = 'kernelgrad' 
overwrite_results = True # overwrites results even when the result file exists, skips the execution otherwise

if user == 'roberto':
    # definir sc
    import findspark
    findspark.init()
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setMaster("local[4]").setAppName("My app").set("spark.executor.memory", "2g"))
    sc = SparkContext(conf = conf)
    import common.lib.svm_utils as SVM_UTILS
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    from math import sqrt
    %matplotlib inline
    # Se importan las funciones
    from pyspark.mllib.util import MLUtils
    from common.lib.IRWLSUtils import *
    Npartitions = 12
    Samplefraction = 0.05

if user == 'navia':
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/IRWLSUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/KernelUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/ResultsUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/SGMAUtils.py")   
    import svm_utils as SVM_UTILS
    #from SGMAUtils import SGMA
    from IRWLSUtils import loadFile, train_SGMA_IRWLS, train_random_IRWLS, train_hybrid_SGMA_IRWLS, train_kmeans_IRWLS
    #from ResultsUtils import compute_AUCs
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    from pyspark.mllib.linalg import SparseVector, DenseVector
    import pickle
    from math import sqrt
    from pyspark.mllib.util import MLUtils
    %matplotlib inline
    Npartitions = -99 # cuando es menor que 0, no se aplica y se deja libre
    Samplefraction = 0.05

# 0 = Ripley
# 1 = Kwok
# 2 = Twonorm
# 3 = Waveform
# 4 = Adult
# 5 = SUSY
# 6 = KddCup1999, requiere preprocesado...
# 7 = Higgs
# 8 = Mnist

datasets = [0, 1, 2, 3, 4, 5, 6, 7, 8]
folds = [0, 1, 2, 3, 4]
dataset_names = ['Ripley', 'Kwok', 'Twonorm', 'Waveform', 'Adult', 'Susy', 'KddCup1999', 'Higgs', 'Mnist']
Niters = [50, 100, 200]
NCs = [5, 10, 25, 50, 100, 200]
modelos = ['hybridgrad', 'kernelgrad', 'SGMA_IRWLS', 'LinearSVM', 'Logistic', 'random_IRWLS', 'hybrid_IRWLS', 'Kmeans_IRWLS']

datasets = [6]
folds = [0]
modelos = ['SGMA_IRWLS']
Niters = [300]
NCs = [75]
Samplefraction = 0.05
fsigma = 1.0

for modelo in modelos:
    for kdataset in datasets:
        for kfold in folds:
            for Niter in Niters:
                C = 100.0
                name_dataset = dataset_names[kdataset]
                
                #####################################################################################################################
                # DATA LOADING: the result of this part must always be three RDDs for train, val and test, containing labelled points.
                #####################################################################################################################
                if kdataset in [0, 1, 2, 3]: # loading from .mat
                    x_tr, y_tr, x_val, y_val, x_tst, y_tst = SVM_UTILS.load_data(kdataset, kfold)
                    NI = x_tr.shape[1]
                    sigma = fsigma * sqrt(NI)
                    
                    if Npartitions > 0:
                        XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Npartitions).map(lambda x: LabeledPoint(x[0], x[1:]))
                        XvalRDD = sc.parallelize(np.hstack((y_val, x_val)), Npartitions).map(lambda x: LabeledPoint(x[0], x[1:]))
                        XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Npartitions).map(lambda x: LabeledPoint(x[0], x[1:]))
                    else:
                        XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr))).map(lambda x: LabeledPoint(x[0], x[1:]))
                        XvalRDD = sc.parallelize(np.hstack((y_val, x_val))).map(lambda x: LabeledPoint(x[0], x[1:]))
                        XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst))).map(lambda x: LabeledPoint(x[0], x[1:]))
                       
                        
                if kdataset in [4]: # loading libsvm format
                    if kdataset == 4:
                        dimensions = 123                       
                        print "Loading " + name_dataset
                        XtrRDD = loadFile('./data/' + name_dataset.lower() + '_train',sc,dimensions,Npartitions)
                        XvalRDD = loadFile('./data/' + name_dataset.lower() + '_val',sc,dimensions,Npartitions)
                        XtstRDD = loadFile('./data/' + name_dataset.lower() + '_test',sc,dimensions,Npartitions)
                        sigma = fsigma * np.sqrt(dimensions)

                        #import code
                        #code.interact(local=locals())

                        #XvalRDD = loadFile('./data/' + name_dataset.lower() + '_val',sc,dimensions,Npartitions)
                        #XtstRDD = loadFile('./data/' + name_dataset.lower() + '_test',sc,dimensions,Npartitions)
                        #sigma = fsigma * np.sqrt(dimensions)

                    
                if kdataset in [5, 8]: # loading in libsvm format and splitting RDD
                    if kdataset == 5:
                        filename = 'file:///export/usuarios01/navia/spark/SVM_spark/data/SUSY.txt'
                        #filename = 'file:///export/usuarios01/navia/spark/SVM_spark/data/minisusy.txt'
                        dimensions = 18
                        rawdata = MLUtils.loadLibSVMFile(sc, filename)
                        # Labelled points are sparse, we map the values
                        rawdata = rawdata.map(lambda x: LabeledPoint(x.label, DenseVector((x.features).toArray())  ))
                        
                    if kdataset == 8:
                        dimensions = 784                       
                        print "Loading " + name_dataset
                        filename = 'file:///export/g2pi/SPARK/data/mnist8m.scale'
                        rawdata = MLUtils.loadLibSVMFile(sc, filename)
                        # Labelled points are sparse, we map the values
                        rawdata = rawdata.map(lambda x: LabeledPoint(x.label, DenseVector((x.features).toArray())  ))
                        # mapping labels to binary
                        rawdata = rawdata.map(SVM_UTILS.multiclass2binary)
                        
                        #import code
                        #code.interact(local=locals())

                    XtrRDD, XvalRDD, XtstRDD = rawdata.randomSplit(weights=[0.7, 0.1, 0.2], seed=1234)
                    sigma = fsigma * np.sqrt(dimensions)

                if kdataset in [6]: # loading as text, transforming variables, converting to labelledpoint and splitting RDD into train, val and test
                    if kdataset == 6:
                        filename = 'file:///export/g2pi/SPARK/data/kddcup1999_tr'
                        XtrRDD = sc.textFile(filename)
                        XtrRDD = XtrRDD.map(SVM_UTILS.text2labeled)

                        filename = 'file:///export/g2pi/SPARK/data/kddcup1999_val'
                        XvalRDD = sc.textFile(filename)
                        XvalRDD = XvalRDD.map(SVM_UTILS.text2labeled)

                        filename = 'file:///export/g2pi/SPARK/data/kddcup1999_tst'
                        XtstRDD = sc.textFile(filename)
                        XtstRDD = XtstRDD.map(SVM_UTILS.text2labeled)

                        XtrRDD.cache()
                        XvalRDD.cache()
                        XtstRDD.cache()
                        sigma = fsigma * np.sqrt(len(XtstRDD.take(1)[0].features))

                if kdataset in [7]: # loading as text, transforming to labelledpoint
                    if kdataset == 7:
                        
                        filename = 'file:///export/g2pi/SPARK/data/higgs_tr.txt'
                        XtrRDD = sc.textFile(filename)
                        XtrRDD = XtrRDD.map(SVM_UTILS.text2labeled)

                        filename = 'file:///export/g2pi/SPARK/data/higgs_val.txt'
                        XvalRDD = sc.textFile(filename)
                        XvalRDD = XvalRDD.map(SVM_UTILS.text2labeled)
                                                
                        filename = 'file:///export/g2pi/SPARK/data/higgs_tst.txt'
                        XtstRDD = sc.textFile(filename)
                        XtstRDD = XtstRDD.map(SVM_UTILS.text2labeled)
                        sigma = fsigma * np.sqrt(len(XtstRDD.take(1)[0].features))

                        XtrRDD.cache()
                        XvalRDD.cache()
                        XtstRDD.cache()

                #################################   END LOADING DATA ##########################################################
                
                for NC in NCs:
                    print "Dataset = %s, modelo = %s, kfold = %d, Niter = %d, NC = %d" % (name_dataset, modelo, kfold, Niter, NC)
                    filename = './results/dataset_' + str(kdataset) + '_modelo_' + modelo + '_NC_' + str(NC) + '_Niter_' + str(Niter) + '_kfold_' + str(kfold) + '.pkl'
                    #import code
                    #code.interact(local=locals())
                    try:
                        f = open(filename,'r')
                        f.close()
                        file_exists = True
                    except:
                        file_exists = False
                        pass
                    execute = False
                    if file_exists:
                        if overwrite_results:
                            execute = True
                    else:
                        execute = True                                  
                    if execute:

                        if modelo == 'hybridgrad':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_hybridSVM(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter, Samplefraction)

                        if modelo == 'kernelgrad':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_kernelgrad(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter, Samplefraction)

                        if modelo == 'SGMA_IRWLS':
                            auc_tr, auc_val, auc_tst, exe_time = train_SGMA_IRWLS(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter)

                        if modelo == 'LinearSVM':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_linear_SVM(XtrRDD, XvalRDD, XtstRDD)
                                                
                        if modelo == 'Logistic':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_logistic(XtrRDD, XvalRDD, XtstRDD)

                        if modelo == 'random_IRWLS':
                            auc_tr, auc_val, auc_tst, exe_time = train_random_IRWLS(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter)

                        if modelo == 'hybrid_IRWLS':
                            auc_tr, auc_val, auc_tst, exe_time = train_hybrid_SGMA_IRWLS(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter)

                        if modelo == 'Kmeans_IRWLS':
                            auc_tr, auc_val, auc_tst, exe_time = train_kmeans_IRWLS(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter)

                        print "Dataset = %s, modelo = %s, kfold = %d, Niter = %d, NC = %d" % (name_dataset, modelo, kfold, Niter, NC)
                        print "AUCtr = %f, AUCval = %f, AUCtst = %f" % (auc_tr, auc_val, auc_tst)
                        print "Elapsed minutes = %f" % (exe_time / 60.0)

                        with open(filename, 'w') as f:
                            pickle.dump([auc_tr, auc_val, auc_tst, exe_time], f)


Py4JJavaError: An error occurred while calling o1290.addFile.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:234)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
py4j.Gateway.invoke(Gateway.java:214)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:79)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:68)
py4j.GatewayConnection.run(GatewayConnection.java:209)
java.lang.Thread.run(Thread.java:745)

